# Phase 3: Object Detection Performance Evaluation
Training and comparing YOLOv8 and RT-DETR on BDD100K with outdoor augmentations.

**Metrics**: mAP@0.5, mAP@0.5:0.95, Precision, Recall, FPS
**Goal**: Determine best detection model and impact of outdoor augmentation

**IMPORTANT**: Set Runtime > Change runtime type > **GPU** (T4) before running!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
RESULTS_DIR = f'{PROJECT_DIR}/results/phase3'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Clone repo and download datasets to LOCAL disk (fast SSD, not Drive)
%cd /content
!rm -rf computer_vision_expirement
!git clone https://github.com/Ib-Programmer/computer_vision_expirement.git
%cd computer_vision_expirement
!pip install -q ultralytics albumentations pyyaml

# Download BDD100K (images + labels) and convert labels to YOLO format
print('\n--- Downloading BDD100K to local disk ---')
!python scripts/download_datasets.py bdd100k
print('\n--- Converting BDD100K labels to YOLO format ---')
!python scripts/preprocess_data.py bdd100k

DATASETS_DIR = '/content/computer_vision_expirement/datasets'
print(f'\nDatasets ready at: {DATASETS_DIR}')
print(f'Results will be saved to Drive: {RESULTS_DIR}')

## 3.1 Verify GPU & Dataset

In [ ]:
import torch
import glob

# GPU check — set DEVICE for all training cells
print('=' * 60)
print('ENVIRONMENT CHECK')
print('=' * 60)
if torch.cuda.is_available():
    DEVICE = 0  # GPU index
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')
    print('Training will use GPU (~1-2 hours for 50 epochs)')
else:
    DEVICE = 'cpu'
    print('WARNING: No GPU detected!')
    print('Go to Runtime > Change runtime type > T4 GPU')
    print('Training on CPU is possible but VERY slow (~40+ hours).')
    print('If your GPU quota is exhausted, try again tomorrow or use Colab Pro.')

print(f'\nDEVICE = {DEVICE}')

# Verify BDD100K YOLO dataset
yolo_dir = f'{DATASETS_DIR}/bdd100k_yolo'
print(f'\nDataset directory: {yolo_dir}')

for split in ['train', 'val']:
    img_dir = f'{yolo_dir}/{split}/images'
    lbl_dir = f'{yolo_dir}/{split}/labels'
    if os.path.exists(img_dir) and os.path.exists(lbl_dir):
        n_imgs = len(glob.glob(f'{img_dir}/*.jpg'))
        n_lbls = len(glob.glob(f'{lbl_dir}/*.txt'))
        non_empty = sum(1 for f in glob.glob(f'{lbl_dir}/*.txt') if os.path.getsize(f) > 0)
        print(f'  {split}: {n_imgs} images, {n_lbls} labels ({non_empty} with objects)')
    else:
        print(f'  {split}: MISSING! Re-run the setup cell above.')

# Show sample label to confirm format is correct
sample_labels = glob.glob(f'{yolo_dir}/train/labels/*.txt')[:5]
for lbl_path in sample_labels:
    with open(lbl_path) as f:
        content = f.read().strip()
    if content:
        print(f'\nSample label ({os.path.basename(lbl_path)}):')
        for line in content.split('\n')[:3]:
            print(f'  {line}')
        break

## 3.2 Prepare Dataset Config

In [ ]:
import yaml

# Create dataset.yaml pointing to YOLO-formatted BDD100K
dataset_config = {
    'path': f'{DATASETS_DIR}/bdd100k_yolo',
    'train': 'train/images',
    'val': 'val/images',
    'names': {
        0: 'pedestrian',
        1: 'rider',
        2: 'car',
        3: 'truck',
        4: 'bus',
        5: 'train',
        6: 'motorcycle',
        7: 'bicycle',
        8: 'traffic light',
        9: 'traffic sign'
    }
}

with open('dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print('dataset.yaml created:')
with open('dataset.yaml') as f:
    print(f.read())

## 3.3 Apply Outdoor Augmentation Patch
Patches Ultralytics' built-in Albumentations class to add outdoor-specific degradations:
- **Fog** (30% chance)
- **Rain** (30% chance)
- **Motion blur** (30% chance)
- **Low-light** (30% chance)
- **Combined effects** (15% chance: fog+blur or rain+dark)

In [ ]:
import albumentations as A
from ultralytics.data.augment import Albumentations

# Save original __init__ so we can restore it for baseline training
_original_alb_init = Albumentations.__init__

def _outdoor_init(self, p=1.0):
    """Patch Albumentations with outdoor-specific augmentations."""
    self.p = p
    T = [
        # Ultralytics defaults (light)
        A.Blur(blur_limit=3, p=0.01),
        A.CLAHE(p=0.01),
        # Outdoor degradations (30% each)
        A.RandomFog(fog_coef_lower=0.2, fog_coef_upper=0.6, alpha_coef=0.08, p=0.3),
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1,
                     drop_color=(200, 200, 200), blur_value=3, brightness_coefficient=0.7, p=0.3),
        A.MotionBlur(blur_limit=(7, 15), p=0.3),
        A.RandomBrightnessContrast(brightness_limit=(-0.5, -0.1), contrast_limit=(-0.3, 0.0), p=0.3),
        # Combined effects (15%)
        A.OneOf([
            A.Compose([A.RandomFog(fog_coef_lower=0.2, fog_coef_upper=0.5, p=1.0),
                       A.MotionBlur(blur_limit=(5, 11), p=1.0)]),
            A.Compose([A.RandomRain(slant_lower=-10, slant_upper=10, p=1.0),
                       A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.1), p=1.0)]),
        ], p=0.15),
    ]
    self.transform = A.Compose(T, bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
    print(f'Outdoor augmentations: fog/rain/blur/dark(30%) + combined(15%)')

# Apply outdoor augmentation patch
Albumentations.__init__ = _outdoor_init
print('Outdoor augmentation patch applied to Ultralytics!')

## 3.4 Train YOLOv8n (Outdoor Augmented)

In [ ]:
from ultralytics import YOLO

print('=' * 60)
print(f'Training YOLOv8n WITH outdoor augmentations (device={DEVICE})')
print('=' * 60)

model_aug = YOLO('yolov8n.pt')
results_aug = model_aug.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    device=DEVICE,
    project=RESULTS_DIR,
    name='yolov8n_outdoor_aug',
    patience=10,
    save=True,
    plots=True
)

## 3.5 Train YOLOv8n (Baseline - No Outdoor Augmentation)

In [ ]:
# Restore default Albumentations (no outdoor effects) for fair baseline
import importlib
import ultralytics.data.augment
importlib.reload(ultralytics.data.augment)
from ultralytics.data.augment import Albumentations

print('=' * 60)
print(f'Training YOLOv8n WITHOUT outdoor augmentations (device={DEVICE})')
print('=' * 60)

model_base = YOLO('yolov8n.pt')
results_base = model_base.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    device=DEVICE,
    project=RESULTS_DIR,
    name='yolov8n_baseline',
    patience=10,
    save=True,
    plots=True
)

## 3.6 Train RT-DETR (Outdoor Augmented)

In [ ]:
# Re-apply outdoor augmentation patch for RT-DETR
from ultralytics.data.augment import Albumentations
Albumentations.__init__ = _outdoor_init

print('=' * 60)
print(f'Training RT-DETR WITH outdoor augmentations (device={DEVICE})')
print('=' * 60)

model_rtdetr = YOLO('rtdetr-l.pt')
results_rtdetr = model_rtdetr.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=8,
    device=DEVICE,
    project=RESULTS_DIR,
    name='rtdetr_outdoor_aug',
    patience=10,
    save=True,
    plots=True
)

## 3.7 Evaluate All Models

In [ ]:
import pandas as pd
import time
import numpy as np

models = {
    'YOLOv8n_outdoor_aug': f'{RESULTS_DIR}/yolov8n_outdoor_aug/weights/best.pt',
    'YOLOv8n_baseline': f'{RESULTS_DIR}/yolov8n_baseline/weights/best.pt',
    'RT-DETR_outdoor_aug': f'{RESULTS_DIR}/rtdetr_outdoor_aug/weights/best.pt',
}

results_table = []

for name, weights_path in models.items():
    if not os.path.exists(weights_path):
        print(f'[SKIP] {name}: weights not found')
        continue

    print(f'\nValidating: {name}')
    model = YOLO(weights_path)
    val_results = model.val(data='dataset.yaml')

    # Measure inference speed
    dummy_img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
    for _ in range(5):  # warmup
        model(dummy_img, verbose=False)
    times = []
    for _ in range(30):
        t0 = time.time()
        model(dummy_img, verbose=False)
        times.append((time.time() - t0) * 1000)

    results_table.append({
        'Model': name,
        'mAP@0.5': round(val_results.box.map50, 4),
        'mAP@0.5:0.95': round(val_results.box.map, 4),
        'Precision': round(val_results.box.mp, 4),
        'Recall': round(val_results.box.mr, 4),
        'Latency_ms': round(np.mean(times), 1),
        'FPS': round(1000 / np.mean(times), 1),
    })

if results_table:
    df = pd.DataFrame(results_table)
    print('\n' + '=' * 60)
    print('DETECTION RESULTS: Baseline vs Outdoor Augmented')
    print('=' * 60)
    print(df.to_string(index=False))
    df.to_csv(f'{RESULTS_DIR}/detection_comparison.csv', index=False)
    print(f'\nSaved to: {RESULTS_DIR}/detection_comparison.csv')
else:
    print('No models to evaluate. Train the models first.')

## 3.8 Visual Results

In [ ]:
import matplotlib.pyplot as plt
import cv2
import glob

# Use best outdoor-augmented model
best_path = f'{RESULTS_DIR}/yolov8n_outdoor_aug/weights/best.pt'
if not os.path.exists(best_path):
    best_path = f'{RESULTS_DIR}/yolov8n_baseline/weights/best.pt'

if os.path.exists(best_path):
    model = YOLO(best_path)

    # Get sample images from BDD100K val set
    test_imgs = glob.glob(f'{DATASETS_DIR}/bdd100k_yolo/val/images/*.jpg')[:6]

    if test_imgs:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        for idx, img_path in enumerate(test_imgs):
            results = model(img_path, verbose=False)
            annotated = results[0].plot()
            ax = axes[idx // 3][idx % 3]
            ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
            ax.set_title(os.path.basename(img_path), fontsize=9)
            ax.axis('off')

        plt.suptitle('YOLOv8 Detection Results (Outdoor Augmented)', fontsize=16)
        plt.tight_layout()
        plt.savefig(f'{RESULTS_DIR}/detection_samples.png', dpi=150, bbox_inches='tight')
        plt.show()
    else:
        print('No test images found.')
else:
    print('Train the models first to see results.')

## 3.9 Cross-Condition Robustness Evaluation
Test the best model's detection performance under simulated adverse outdoor conditions.
This validates whether outdoor augmentation during training improves robustness.

In [ ]:
import albumentations as A
import pandas as pd
import numpy as np
import cv2
import glob
import time

# Define outdoor degradation conditions
conditions = {
    'Clean (original)': None,
    'Fog (heavy)': A.Compose([A.RandomFog(fog_coef_lower=0.4, fog_coef_upper=0.8, alpha_coef=0.08, p=1.0)]),
    'Rain (heavy)': A.Compose([A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1,
                                             drop_color=(200, 200, 200), blur_value=5, brightness_coefficient=0.6, p=1.0)]),
    'Low-light': A.Compose([A.RandomBrightnessContrast(brightness_limit=(-0.6, -0.3), contrast_limit=(-0.3, 0.0), p=1.0)]),
    'Motion blur': A.Compose([A.MotionBlur(blur_limit=(11, 21), p=1.0)]),
    'Fog + Dark': A.Compose([A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, p=1.0),
                              A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.1), p=1.0)]),
}

# Evaluate both augmented and baseline models under each condition
model_paths = {
    'YOLOv8n_outdoor_aug': f'{RESULTS_DIR}/yolov8n_outdoor_aug/weights/best.pt',
    'YOLOv8n_baseline': f'{RESULTS_DIR}/yolov8n_baseline/weights/best.pt',
}

val_images = sorted(glob.glob(f'{DATASETS_DIR}/bdd100k_yolo/val/images/*.jpg'))[:100]
print(f'Evaluating {len(val_images)} images under {len(conditions)} conditions...\n')

cross_results = []

for model_name, model_path in model_paths.items():
    if not os.path.exists(model_path):
        print(f'[SKIP] {model_name}: weights not found')
        continue
    
    model = YOLO(model_path)
    
    for cond_name, transform in conditions.items():
        detections = 0
        total_conf = 0
        n_images = 0
        
        for img_path in val_images:
            img = cv2.imread(img_path)
            if img is None:
                continue
            
            # Apply degradation
            if transform is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_aug = transform(image=img_rgb)['image']
                img = cv2.cvtColor(img_aug, cv2.COLOR_RGB2BGR)
            
            results = model(img, verbose=False, conf=0.25)
            n_boxes = len(results[0].boxes)
            detections += n_boxes
            if n_boxes > 0:
                total_conf += results[0].boxes.conf.mean().item()
            n_images += 1
        
        avg_det = detections / max(n_images, 1)
        avg_conf = total_conf / max(n_images, 1)
        
        cross_results.append({
            'Model': model_name,
            'Condition': cond_name,
            'Avg_Detections': round(avg_det, 2),
            'Avg_Confidence': round(avg_conf, 4),
            'Total_Objects': detections,
        })
        print(f'  {model_name} | {cond_name}: {avg_det:.1f} det/img, conf={avg_conf:.3f}')

if cross_results:
    cross_df = pd.DataFrame(cross_results)
    print('\n' + '=' * 70)
    print('CROSS-CONDITION ROBUSTNESS RESULTS')
    print('=' * 70)
    print(cross_df.to_string(index=False))
    cross_df.to_csv(f'{RESULTS_DIR}/cross_condition_robustness.csv', index=False)
    print(f'\nSaved to: {RESULTS_DIR}/cross_condition_robustness.csv')
    
    # Show improvement from outdoor augmentation
    print('\n--- Augmentation Benefit Analysis ---')
    for cond in conditions:
        aug_row = [r for r in cross_results if r['Model'] == 'YOLOv8n_outdoor_aug' and r['Condition'] == cond]
        base_row = [r for r in cross_results if r['Model'] == 'YOLOv8n_baseline' and r['Condition'] == cond]
        if aug_row and base_row:
            delta = aug_row[0]['Avg_Detections'] - base_row[0]['Avg_Detections']
            print(f'  {cond}: augmented has {delta:+.1f} more det/img')

## 3.10 Literature Comparison (Published Benchmarks)
Comparing our results with published detection benchmarks from peer-reviewed papers.

In [ ]:
import pandas as pd

# Published benchmarks: COCO val2017
coco_literature = pd.DataFrame([
    {'Method': 'Faster R-CNN (Ren et al., 2015)', 'Backbone': 'ResNet-101-FPN', 'mAP@0.5': 59.1, 'mAP@0.5:0.95': 36.2, 'FPS': 5.0, 'Source': 'NeurIPS 2015'},
    {'Method': 'SSD300 (Liu et al., 2016)', 'Backbone': 'VGG-16', 'mAP@0.5': 43.1, 'mAP@0.5:0.95': 25.1, 'FPS': 46.0, 'Source': 'ECCV 2016'},
    {'Method': 'SSD512 (Liu et al., 2016)', 'Backbone': 'VGG-16', 'mAP@0.5': 48.5, 'mAP@0.5:0.95': 28.8, 'FPS': 19.0, 'Source': 'ECCV 2016'},
    {'Method': 'RetinaNet (Lin et al., 2017)', 'Backbone': 'ResNet-101-FPN', 'mAP@0.5': 57.5, 'mAP@0.5:0.95': 37.8, 'FPS': 11.0, 'Source': 'ICCV 2017'},
    {'Method': 'DETR (Carion et al., 2020)', 'Backbone': 'ResNet-50', 'mAP@0.5': 58.9, 'mAP@0.5:0.95': 42.0, 'FPS': 28.0, 'Source': 'ECCV 2020'},
    {'Method': 'Deformable DETR (Zhu et al., 2021)', 'Backbone': 'ResNet-50', 'mAP@0.5': 62.3, 'mAP@0.5:0.95': 43.8, 'FPS': 19.0, 'Source': 'ICLR 2021'},
    {'Method': 'YOLOv8n (Jocher et al., 2023)', 'Backbone': 'CSPDarknet', 'mAP@0.5': 52.6, 'mAP@0.5:0.95': 37.3, 'FPS': 400.0, 'Source': 'Ultralytics 2023'},
    {'Method': 'YOLOv8s (Jocher et al., 2023)', 'Backbone': 'CSPDarknet', 'mAP@0.5': 57.6, 'mAP@0.5:0.95': 44.9, 'FPS': 280.0, 'Source': 'Ultralytics 2023'},
    {'Method': 'YOLOv8l (Jocher et al., 2023)', 'Backbone': 'CSPDarknet', 'mAP@0.5': 62.6, 'mAP@0.5:0.95': 52.9, 'FPS': 120.0, 'Source': 'Ultralytics 2023'},
    {'Method': 'RT-DETR-L (Lv et al., 2023)', 'Backbone': 'ResNet-50/HGNetv2', 'mAP@0.5': 72.8, 'mAP@0.5:0.95': 53.0, 'FPS': 114.0, 'Source': 'arXiv 2023'},
])

# Published benchmarks: BDD100K val
bdd_literature = pd.DataFrame([
    {'Method': 'Faster R-CNN', 'mAP@0.5': 30.0, 'Source': 'BDD100K paper (Yu et al., 2020)'},
    {'Method': 'Cascade R-CNN', 'mAP@0.5': 32.5, 'Source': 'BDD100K paper'},
    {'Method': 'YOLOv5s', 'mAP@0.5': 38.9, 'Source': 'Community benchmark'},
    {'Method': 'YOLOv8n', 'mAP@0.5': 40.2, 'Source': 'Community benchmark'},
    {'Method': 'YOLOv8l', 'mAP@0.5': 48.3, 'Source': 'Community benchmark'},
    {'Method': 'RT-DETR-L', 'mAP@0.5': 46.5, 'Source': 'Community benchmark'},
])

print('=' * 80)
print('TABLE 3.1: Object Detection — Published Benchmarks (COCO val2017)')
print('=' * 80)
print(coco_literature.to_string(index=False))

print('\n' + '=' * 80)
print('TABLE 3.2: Object Detection — Published Benchmarks (BDD100K val)')
print('=' * 80)
print(bdd_literature.to_string(index=False))

# Compare our results with literature
if results_table:
    print('\n' + '=' * 80)
    print('TABLE 3.3: Our Results vs Literature (BDD100K val)')
    print('=' * 80)
    our_df = pd.DataFrame(results_table)
    print(our_df[['Model', 'mAP@0.5', 'mAP@0.5:0.95', 'FPS']].to_string(index=False))
    print('\nNote: Our models trained for 50 epochs on BDD100K train split.')
    print('Published benchmarks use full training schedules and larger model variants.')

# Save
coco_literature.to_csv(f'{RESULTS_DIR}/literature_coco_detection.csv', index=False)
bdd_literature.to_csv(f'{RESULTS_DIR}/literature_bdd_detection.csv', index=False)
print(f'\nLiterature tables saved to: {RESULTS_DIR}')

In [ ]:
print(f'\nPhase 3 results saved to: {RESULTS_DIR}')
print('Sections completed:')
print('  3.1-3.6: Model training (YOLOv8n baseline/augmented, RT-DETR)')
print('  3.7-3.8: Evaluation and visualization')
print('  3.9: Cross-condition robustness analysis')
print('  3.10: Literature comparison with published benchmarks')
print('Next: Open Phase4_Face_Recognition.ipynb')